# Create miRNASNP-v3 index for BIGD

Guide to for index file [NGDC](https://bigd.big.ac.cn/standards/dis) Bigsearch system (index.bs)

**index.bs** file structure as following:

```
DB JSON_ENCODED_STRING
ENTRY JSON_ENCODED_STRING
```

In [14]:
import json
import uuid

In [40]:
db_dict = {
    "id": "mirnasnp",
    "title": "miRNASNP-v3",
    "url": "http://bioinfo.life.hust.edu.cn/miRNASNP/",
    "description": "miRNASNP-v3: a comprehensive database for SNPs and disease-related variations in miRNAs and miRNA targets",
    "basicInfo": "In miRNASNP-v3, 46,826 SNPs in human 1,897 pre-miRNAs (2,624 mature miRNAs) and 7,115,796  SNPs in 3'UTRs of 18,151 genes were characterized. Besides, 505,417 disease-related variations (DRVs) from GWAS, ClinVar and COSMIC were identified in miRNA and gene 3'UTR. Gene enrichment of target gain/loss by variations in miRNA seed region was provided.",
    "categories": ["miRNA", "mutation", "disease"],
    "species": ["Homo Sapiens"],
    "updateAt": "2020-09-02 11:11:11"
}

In [44]:
class ENTRY(object):
    def __init__(self):
        self.id = str(uuid.uuid4())
        self.type = "gene"
        self.title = ""
        self.url = ""
        self.dbId = "mirnasnp"
        self.updatedAt = "2020-09-02 11:11:11"
        self.description = ""
        self.basicInfo = ""
        self.species = ["Homo Sapiens"]
        self.attrs = {
            "accession": "",
        }
    def __getattr__(self, attr):
        return self[attr]